In [1]:
import os
import sys
import csv
import wave
import copy
import math

import numpy as np
import pandas as pd

from sklearn.preprocessing import label_binarize
from sklearn.cross_validation import StratifiedKFold, KFold, train_test_split
from sklearn.svm import OneClassSVM, SVC
from sklearn.ensemble import RandomForestClassifier
from sklearn.grid_search import GridSearchCV
from sklearn.manifold import TSNE
from sklearn.decomposition import PCA

from keras.models import Sequential, Model
from keras.layers.core import Dense, Activation
from keras.layers import LSTM, Input
from keras.layers.wrappers import TimeDistributed
from keras.optimizers import SGD, Adam, RMSprop

sys.path.append("../")

from utilities.utils import *

import matplotlib
import matplotlib.pyplot as plt
import matplotlib.ticker as ticker
%matplotlib inline

from IPython.display import clear_output

/usr/local/lib/python3.5/dist-packages/sklearn/cross_validation.py:41: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. Also note that the interface of the new CV iterators are different from that of this module. This module will be removed in 0.20.
  "This module will be removed in 0.20.", DeprecationWarning)
/usr/local/lib/python3.5/dist-packages/sklearn/grid_search.py:42: DeprecationWarning: This module was deprecated in version 0.18 in favor of the model_selection module into which all the refactored classes and functions are moved. This module will be removed in 0.20.
  DeprecationWarning)
Using TensorFlow backend.


In [6]:
batch_size = 64
nb_feat = 34
nb_class = 4
nb_epoch = 80

optimizer = 'Adadelta'

params = Constants()
print(params)

----------------------------------------------------------------------------------------------------
available_emotions            ['ang' 'exc' 'neu' 'sad']                                             
conf_matrix_prefix            iemocap                                                               
framerate                     16000                                                                 
path_to_data                  /home/samarth/emotion_recognition-master/code/utilities/../../data/ses
path_to_features              /home/samarth/emotion_recognition-master/code/utilities/../../data/fea
sessions                      ['Session1', 'Session2', 'Session3', 'Session4', 'Session5']          
types                         {1: <class 'numpy.int8'>, 2: <class 'numpy.int16'>, 4: <class 'numpy.i
----------------------------------------------------------------------------------------------------


In [8]:
import pickle
with open(params.path_to_data + '/../'+'data_collected.pickle', 'rb') as handle:
    data2 = pickle.load(handle)

In [9]:
text = []


for ses_mod in data2:
    text.append(ses_mod['transcription'])

In [10]:
from os import listdir
import random
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Dense, Dropout, Reshape, Merge, BatchNormalization, TimeDistributed, Lambda, Activation, LSTM, Flatten, Convolution1D, GRU, MaxPooling1D
from keras.regularizers import l2
from keras.callbacks import Callback, ModelCheckpoint, EarlyStopping
#from keras import initializers
from keras import backend as K
from keras.optimizers import SGD
from keras.optimizers import Adadelta
from keras.utils import np_utils
from keras.preprocessing import sequence
from keras import optimizers
import numpy as np

In [11]:
MAX_SEQUENCE_LENGTH = 500

tokenizer = Tokenizer()
tokenizer.fit_on_texts(text)

token_tr_X = tokenizer.texts_to_sequences(text)
x_train_text = []

x_train_text = sequence.pad_sequences(token_tr_X, maxlen=MAX_SEQUENCE_LENGTH)

In [12]:
import codecs
EMBEDDING_DIM = 300

word_index = tokenizer.word_index
print('Found %s unique tokens' % len(word_index))

file_loc = params.path_to_data + '../glove.42B.300d.txt'

print (file_loc)

gembeddings_index = {}
with codecs.open(file_loc, encoding='utf-8') as f:
    for line in f:
        values = line.split(' ')
        word = values[0]
        gembedding = np.asarray(values[1:], dtype='float32')
        gembeddings_index[word] = gembedding
#
f.close()
print('G Word embeddings:', len(gembeddings_index))

nb_words = len(word_index) +1
g_word_embedding_matrix = np.zeros((nb_words, EMBEDDING_DIM))
for word, i in word_index.items():
    gembedding_vector = gembeddings_index.get(word)
    if gembedding_vector is not None:
        g_word_embedding_matrix[i] = gembedding_vector
        
print('G Null word embeddings: %d' % np.sum(np.sum(g_word_embedding_matrix, axis=1) == 0))

Found 2736 unique tokens
/home/samarth/emotion_recognition-master/code/utilities/../../data/sessions/../glove.42B.300d.txt
G Word embeddings: 1917494
G Null word embeddings: 90


In [13]:
Y=[]
for ses_mod in data2:
    Y.append(ses_mod['emotion'])
    
Y = to_categorical(Y)

Y.shape

(4936, 4)

In [32]:
model = Sequential()
#model.add(Embedding(2737, 128, input_length=MAX_SEQUENCE_LENGTH))
model.add(Embedding(nb_words,
                    EMBEDDING_DIM,
                    weights = [g_word_embedding_matrix],
                    input_length = MAX_SEQUENCE_LENGTH,
                    trainable = True))
model.add(Convolution1D(256, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(128, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(64, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Convolution1D(32, 3, border_mode='same'))
model.add(Dropout(0.2))
model.add(Activation('relu'))
model.add(Flatten())
model.add(Dropout(0.2))
model.add(Dense(256))
model.add(Activation('relu')) 
model.add(Dropout(0.2))
model.add(Dense(4))
model.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model.summary()
model.save(params.path_to_data + '/../'+'text_model.h5')

print("Model1 Built")

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(256, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(128, 3, padding="same")`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(64, 3, padding="same")`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv1D` call to the Keras 2 API: `Conv1D(32, 3, padding="same")`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 300)          821100    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 500, 256)          230656    
_________________________________________________________________
dropout_19 (Dropout)         (None, 500, 256)          0         
_________________________________________________________________
activation_21 (Activation)   (None, 500, 256)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 500, 128)          98432     
_________________________________________________________________
dropout_20 (Dropout)         (None, 500, 128)          0         
_________________________________________________________________
activation_22 (Activation)   (None, 500, 128)          0         
__________

In [33]:
filepath="text_model_weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

hist = model.fit(x_train_text, Y, shuffle=True,
                 batch_size=batch_size, nb_epoch=30, verbose=1, 
                 validation_split=0.2, callbacks= callbacks_list)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 6s 2ms/step - loss: 1.3691 - acc: 0.3361 - val_loss: 1.3648 - val_acc: 0.3684
Epoch 2/30
3948/3948 [==============================] - 5s 1ms/step - loss: 1.3333 - acc: 0.3607 - val_loss: 1.2545 - val_acc: 0.4352
Epoch 3/30
3948/3948 [==============================] - 5s 1ms/step - loss: 1.1745 - acc: 0.4856 - val_loss: 1.0856 - val_acc: 0.5405
Epoch 4/30
3948/3948 [==============================] - 5s 1ms/step - loss: 0.8973 - acc: 0.6459 - val_loss: 1.0764 - val_acc: 0.5668
Epoch 5/30
3948/3948 [==============================] - 5s 1ms/step - loss: 0.6847 - acc: 0.7391 - val_loss: 1.0267 - val_acc: 0.6053
Epoch 6/30
3948/3948 [==============================] - 5s 1ms/step - loss: 0.5524 - acc: 0.7893 - val_loss: 1.1023 - val_acc: 0.6053
Epoch 7/30
3948/3948 [==============================] - 5s 1ms/step - loss: 0.4580 - acc: 0.8321 - val_loss: 1.1987 - val_acc: 0.5789
Epoch 8/30
3948

In [23]:
from keras.models import load_model

model = load_model(params.path_to_data + '/../'+'text_model.h5')
model.load_weights('text_model_weights-07-0.66.hdf5')
model.summary()
model.layers.pop()
model.layers.pop()
model.layers.pop()
model.layers.pop()
model.summary()


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_3 (Embedding)      (None, 500, 300)          821100    
_________________________________________________________________
conv1d_1 (Conv1D)            (None, 500, 256)          230656    
_________________________________________________________________
dropout_1 (Dropout)          (None, 500, 256)          0         
_________________________________________________________________
activation_1 (Activation)    (None, 500, 256)          0         
_________________________________________________________________
conv1d_2 (Conv1D)            (None, 500, 128)          98432     
_________________________________________________________________
dropout_2 (Dropout)          (None, 500, 128)          0         
_________________________________________________________________
activation_2 (Activation)    (None, 500, 128)          0         
__________

In [24]:
x_train_mocap = []
from sklearn.preprocessing import normalize
counter = 0
for ses_mod in data2:
    x_head = ses_mod['mocap_head']
    if(x_head.shape != (200,18)):
        x_head = np.zeros((200,18))   
    x_head[np.isnan(x_head)]=0
    x_hand = ses_mod['mocap_hand']
    if(x_hand.shape != (200,6)):
        x_hand = np.zeros((200,6))   
    x_hand[np.isnan(x_hand)]=0
    x_rot = ses_mod['mocap_rot']
    if(x_rot.shape != (200,165)):
        x_rot = np.zeros((200,165))   
    x_rot[np.isnan(x_rot)]=0
    x_mocap = np.concatenate((x_head, x_hand), axis=1)
    x_mocap = np.concatenate((x_mocap, x_rot), axis=1)
    x_train_mocap.append( x_mocap )
    
x_train_mocap = np.array(x_train_mocap)
x_train_mocap = x_train_mocap.reshape(-1,200,189,1)
x_train_mocap.shape

(4936, 200, 189, 1)

In [29]:

from keras.layers import Conv2D

model_mocap = Sequential()
model_mocap.add(Conv2D(32, 3, strides=(2, 2), border_mode='same', input_shape=(200, 189, 1)))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(64, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Conv2D(128, 3, strides=(2, 2), border_mode='same'))
model_mocap.add(Dropout(0.2))
model_mocap.add(Activation('relu'))
model_mocap.add(Flatten())
model_mocap.add(Dense(256))
model_mocap.add(Activation('relu')) 
model_mocap.add(Dropout(0.2))
model_mocap.add(Dense(4))
model_mocap.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_mocap.compile(loss='categorical_crossentropy',optimizer='adam' ,metrics=['acc'])

## compille it here according to instructions

#model.compile()
model_mocap.summary()
model_mocap.save(params.path_to_data + '/../'+'mocap_model.h5')

/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:5: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(32, 3, input_shape=(200, 189,..., strides=(2, 2), padding="same")`
  """
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:8: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:11: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(64, 3, padding="same", strides=(2, 2))`
  # This is added back by InteractiveShellApp.init_path()
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:14: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="same", strides=(2, 2))`
  
/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:17: UserWarning: Update your `Conv2D` call to the Keras 2 API: `Conv2D(128, 3, padding="same", strides=(2, 2))`


_________________________________________________________________
Layer (type)                 Output Shape              Param #   
conv2d_6 (Conv2D)            (None, 100, 95, 32)       320       
_________________________________________________________________
dropout_13 (Dropout)         (None, 100, 95, 32)       0         
_________________________________________________________________
activation_14 (Activation)   (None, 100, 95, 32)       0         
_________________________________________________________________
conv2d_7 (Conv2D)            (None, 50, 48, 64)        18496     
_________________________________________________________________
dropout_14 (Dropout)         (None, 50, 48, 64)        0         
_________________________________________________________________
activation_15 (Activation)   (None, 50, 48, 64)        0         
_________________________________________________________________
conv2d_8 (Conv2D)            (None, 25, 24, 64)        36928     
__________

In [30]:
filepath="mocap_model_weights-{epoch:02d}-{val_acc:.2f}.hdf5"
checkpoint = ModelCheckpoint(filepath, monitor='val_acc', verbose=1, save_best_only=True, mode='max')
callbacks_list = [checkpoint]

hist = model_mocap.fit(x_train_mocap, Y, shuffle=True,
                 batch_size=batch_size, nb_epoch=80, verbose=1, 
                 validation_split=0.2, callbacks= callbacks_list)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/80
3948/3948 [==============================] - 9s 2ms/step - loss: 2.4900 - acc: 0.3062 - val_loss: 1.3679 - val_acc: 0.3694
Epoch 2/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.3732 - acc: 0.3397 - val_loss: 1.3681 - val_acc: 0.3694
Epoch 3/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.3665 - acc: 0.3399 - val_loss: 1.3618 - val_acc: 0.3694
Epoch 4/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.3618 - acc: 0.3412 - val_loss: 1.3485 - val_acc: 0.3694
Epoch 5/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.3464 - acc: 0.3412 - val_loss: 1.3312 - val_acc: 0.3694
Epoch 6/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.3163 - acc: 0.3678 - val_loss: 1.2131 - val_acc: 0.4990
Epoch 7/80
3948/3948 [==============================] - 8s 2ms/step - loss: 1.2577 - acc: 0.3967 - val_loss: 1.2562 - val_acc: 0.4980
Epoch 8/80
3948

3948/3948 [==============================] - 8s 2ms/step - loss: 0.9545 - acc: 0.5585 - val_loss: 1.8052 - val_acc: 0.4170
Epoch 33/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.9584 - acc: 0.5631 - val_loss: 1.6114 - val_acc: 0.4393
Epoch 34/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.9514 - acc: 0.5704 - val_loss: 1.8041 - val_acc: 0.4190
Epoch 35/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.9357 - acc: 0.5849 - val_loss: 1.7027 - val_acc: 0.4676
Epoch 36/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.9335 - acc: 0.5897 - val_loss: 1.8748 - val_acc: 0.4332
Epoch 37/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.8970 - acc: 0.6082 - val_loss: 2.0558 - val_acc: 0.4383
Epoch 38/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.8975 - acc: 0.6079 - val_loss: 1.9984 - val_acc: 0.4534
Epoch 39/80
3948/3948 [==============================] - 8s 2ms/ste

Epoch 65/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.6353 - acc: 0.7356 - val_loss: 2.9719 - val_acc: 0.4211
Epoch 66/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.5984 - acc: 0.7546 - val_loss: 3.5495 - val_acc: 0.4383
Epoch 67/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.6168 - acc: 0.7452 - val_loss: 3.0257 - val_acc: 0.4565
Epoch 68/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.5929 - acc: 0.7452 - val_loss: 3.5591 - val_acc: 0.4494
Epoch 69/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.5714 - acc: 0.7606 - val_loss: 3.3374 - val_acc: 0.4524
Epoch 70/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.5592 - acc: 0.7632 - val_loss: 4.1578 - val_acc: 0.4332
Epoch 71/80
3948/3948 [==============================] - 8s 2ms/step - loss: 0.5487 - acc: 0.7730 - val_loss: 3.8185 - val_acc: 0.4626
Epoch 72/80
3948/3948 [==============================] 

In [34]:
from keras.models import load_model

modelt = load_model(params.path_to_data + '/../'+'text_model.h5')
modelt.load_weights('text_model_weights-14-0.65.hdf5')
modelt.summary()
modelt.layers.pop()
modelt.layers.pop()
modelt.layers.pop()
modelt.layers.pop()
modelt.summary()

modelm = load_model(params.path_to_data + '/../'+'mocap_model.h5')
modelm.load_weights('mocap_model_weights-09-0.51.hdf5')
modelm.summary()
modelm.layers.pop()
modelm.layers.pop()
modelm.layers.pop()
modelm.layers.pop()
modelm.summary()

_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_4 (Embedding)      (None, 500, 300)          821100    
_________________________________________________________________
conv1d_5 (Conv1D)            (None, 500, 256)          230656    
_________________________________________________________________
dropout_19 (Dropout)         (None, 500, 256)          0         
_________________________________________________________________
activation_21 (Activation)   (None, 500, 256)          0         
_________________________________________________________________
conv1d_6 (Conv1D)            (None, 500, 128)          98432     
_________________________________________________________________
dropout_20 (Dropout)         (None, 500, 128)          0         
_________________________________________________________________
activation_22 (Activation)   (None, 500, 128)          0         
__________

In [36]:
model_combined = Sequential()
model_combined.add(Merge([modelt, modelm], mode='concat'))

model_combined.add(Dense(256))
model_combined.add(Activation('relu'))

model_combined.add(Dense(4))
model_combined.add(Activation('softmax'))

#sgd = optimizers.SGD(lr=0.001, decay=1e-6, momentum=0.9, nesterov=True)
model_combined.compile(loss='categorical_crossentropy',optimizer='Adam' ,metrics=['acc'])


/usr/local/lib/python3.5/dist-packages/ipykernel_launcher.py:2: UserWarning: The `Merge` layer is deprecated and will be removed after 08/2017. Use instead layers from `keras.layers.merge`, e.g. `add`, `concatenate`, etc.
  


In [37]:
hist = model_combined.fit([x_train_text,x_train_mocap], Y, 
                 batch_size=batch_size, nb_epoch=30, verbose=1, 
                 validation_split=0.2)

/usr/local/lib/python3.5/dist-packages/keras/models.py:939: UserWarning: The `nb_epoch` argument in `fit` has been renamed `epochs`.
  warnings.warn('The `nb_epoch` argument in `fit` '


Train on 3948 samples, validate on 988 samples
Epoch 1/30
3948/3948 [==============================] - 14s 3ms/step - loss: 0.9253 - acc: 0.8336 - val_loss: 0.9665 - val_acc: 0.6356
Epoch 2/30
3948/3948 [==============================] - 13s 3ms/step - loss: 0.4494 - acc: 0.8797 - val_loss: 1.0410 - val_acc: 0.6194
Epoch 3/30
3948/3948 [==============================] - 13s 3ms/step - loss: 0.3283 - acc: 0.9005 - val_loss: 1.2678 - val_acc: 0.5962
Epoch 4/30
1280/3948 [========>.....................] - ETA: 8s - loss: 0.3149 - acc: 0.9016

KeyboardInterrupt: 